In [1]:
import os
from dataverse_api import DataverseClient
from msal import ConfidentialClientApplication
from msal_requests_auth.auth import ClientCredentialAuth

import json
import requests
from dataverse_api._api import DataverseAPI
from dataverse_api._metadata_defs import EntityMetadata, StringAttributeMetadata, ManagedProperty, Label


app_id = os.environ["app_id"]
authority_url = os.environ["authority_url"]
client_secret = os.environ["client_secret"]
url = os.environ["url"]  # https://org_identifier.crm.dynamics.com
scopes = [url + "/.default"]


client = ConfidentialClientApplication(
    client_id=app_id,
    client_credential=client_secret,
    authority=authority_url,
)
auth = ClientCredentialAuth(
    client=client,
    scopes=scopes,
)

client = DataverseClient(resource=url, auth=auth)
api = DataverseAPI(client._auth)




In [2]:
### CREATING NEW ENTITY ###



# a = EntityMetadata(
#     description=Label("Table for collecting logs for Azure function runs.", 1033),
#     display_name=Label("Log Runs", 1033),
#     schema_name="test_logruns",
#     display_collection_name=Label("Log Runs", 1033),
#     attributes=[StringAttributeMetadata(
#         description=Label("Function Name", 1033),
#         display_name=Label("Function Name", 1033),
#         schema_name="test_function_name",
#         max_length=100,
#         format_name="Text",
#         is_primary=True,
#         required_level=ManagedProperty(value="None",can_be_changed=True,managed_property_name="canmodifyrequirementlevelsettings")
#     )],
#     ownership_type="UserOwned",
#     is_activity=False,
#     has_notes=False,
#     has_activities=False
# )

# create = requests.post(
#     url="https://org2ae976c6.crm4.dynamics.com/api/data/v9.2/EntityDefinitions",
#     auth=client._auth.auth,
#     headers={"Accept":"application/json"},
#     json=a()
# )

In [2]:
# ESTABLISHING RELATIONSHIP

from dataverse_api._metadata_defs import LookupAttributeMetadata, OneToManyRelationshipMetadata, Label, RequiredLevel, AssociatedMenuConfiguration, CascadeConfiguration

label = Label("Associated TEST ", 1033)

lookup = LookupAttributeMetadata(
    description=label,
    display_name=label,
    schema_name="test_lookupID",
    required_level=RequiredLevel()
)

rel = OneToManyRelationshipMetadata(
    schema_name="test_rel_marcus_logs",
    cascade_configuration=CascadeConfiguration(),
    associated_menu_config=AssociatedMenuConfiguration(
        behavior="UseCollectionName",
        group="Details",
        label=Label("Associated Menu Config Label", 1033),
        order=10000
    ),
    # referenced_attribute="cr3fa_test_value_text",
    referenced_entity="cr3fa_marcus_test",
    referencing_entity="test_logruns",
    lookup=lookup
)


In [6]:
relationship = requests.post(
    url="https://org2ae976c6.crm4.dynamics.com/api/data/v9.2/RelationshipDefinitions",
    auth=client._auth.auth,
    headers={"Accept":"application/json"},
    json=rel()
)

relationship

<Response [204]>

In [5]:
requests.delete(relationship.headers["Location"], auth=client._auth.auth)

KeyError: 'location'

In [16]:
### PERFORMING DEEP INSERT ON PARENT TABLE TO CHILD TABLE

table = client.entity("cr3fa_marcus_test", validate=True)
table._post(
    url=table.schema.entity.name, 
    json={
        "cr3fa_test_pk":"DEEP",
        "test_rel_marcus_logs": [
            {
                "test_function_name":"FUNC1", 
                "new_run_detail":"Warning: Encountered stuff"
            },
            {
                "test_function_name":"FUNC1", 
                "new_run_detail":"Warning: Missing Values"
            }
        ]
    }
)

In [11]:
client.entity("test_logruns").read(select=["test_function_name","new_run_detail","_test_lookupid_value"])

[{'@odata.etag': 'W/"14559105"',
  'test_function_name': 'FUNC1',
  'new_run_detail': 'Warning: Encountered stuff',
  '_test_lookupid_value': 'c41793f8-a267-ee11-9ae7-6045bd8863da',
  'test_logrunsid': 'c51793f8-a267-ee11-9ae7-6045bd8863da'},
 {'@odata.etag': 'W/"14559106"',
  'test_function_name': 'FUNC1',
  'new_run_detail': 'Warning: Missing Values',
  '_test_lookupid_value': 'c41793f8-a267-ee11-9ae7-6045bd8863da',
  'test_logrunsid': 'c61793f8-a267-ee11-9ae7-6045bd8863da'},
 {'@odata.etag': 'W/"14556553"',
  'test_function_name': None,
  'new_run_detail': None,
  '_test_lookupid_value': None,
  'test_logrunsid': '559e40b9-9f67-ee11-9ae7-6045bd886bc5'}]